In [3]:
import os
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet
import base64
import json
from datetime import datetime
# utilizaremos las mismas funciones del laboratorio de claves asimétricas

def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=3854
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )


def crear_documento(autor, titulo, contenido, nombre_archivo="documento.json"):
    """
    Crea un documento en formato JSON para ser firmado.

    Args:
        autor (str): Nombre del autor del documento
        titulo (str): Título del documento
        contenido (str): Contenido del documento
        nombre_archivo (str): Nombre del archivo donde se guardará el documento

    Returns:
        dict: El documento creado
    """
    documento = {
        "autor": autor,
        "titulo": titulo,
        "contenido": contenido,
        "fecha_creacion": datetime.now().isoformat()
    }

    with open(nombre_archivo, "w") as f:
        json.dump(documento, f, indent=4)

    print(f"Documento creado y guardado en: {nombre_archivo}")
    return documento


#NOTA:  revisar que el documento exista en el disco antes de proceder con la actividad!
# función para firmar digitalmente un documento
def firmar_documento(ruta_documento, ruta_firma="firma_digital.bin"):
    """
    Firma digitalmente un ruta_documento utilizando una clave privada RSA.

    Args:
        ruta_clave_privada (str): Ruta al archivo de clave privada PEM
        documento (dict): Documento a firmar
        ruta_firma (str): Ruta donde se guardará la firma

    Returns:
        bytes: La firma digital
    """
    # Cargamos la clave privada
    private_key = cargar_clave_privada()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()


    # Firmamos el mensaje
    firma = private_key.sign(
        mensaje,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    # Guardamos la firma en un archivo
    with open(ruta_firma, "wb") as f:
        f.write(firma)

    print(f"Documento firmado. Firma guardada en: {ruta_firma}")

    # También guardamos la firma en formato base64 para visualización
    firma_base64 = base64.b64encode(firma).decode('utf-8')
    with open(f"{ruta_firma}.b64", "w") as f:
        f.write(firma_base64)

    print(f"Firma en formato base64 guardada en: {ruta_firma}.b64")

    return firma

def verificar_firma(ruta_documento, ruta_firma):
    """
    Verifica la firma digital de un documento utilizando una clave pública RSA.

    Args:
        ruta_clave_publica (str): Ruta al archivo de clave pública PEM
        documento (dict): Documento cuya firma se va a verificar
        ruta_firma (str): Ruta al archivo de firma

    Returns:
        bool: True si la firma es válida, False en caso contrario
    """
    # Cargamos la clave pública
    public_key = cargar_clave_publica()

    # Cargamos la firma
    with open(ruta_firma, "rb") as f:
        firma = f.read()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()


    try:
        # Verificamos la firma
        public_key.verify(
            firma,
            mensaje,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("¡La firma es válida! El documento es auténtico y no ha sido modificado.")
        return True
    except Exception as e:
        print("¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.")
        print(f"Error: {e}")
        return False


# --- Implementación de casos con match ---

def run_case(case_number):
    match case_number:
        case 1:
            print("\n--- Caso 1: Generar claves, crear documento, firmar y verificar (original) ---")
            generar_claves_asimetricas()
            crear_documento(
                "Autor de ejemplo (gabriel rivera )",
                "Declaracion firmada",
                "En este documento oficial listamos las vulnerabilidades de seguridad encontradas en nuestros sistemas\n"
                "no debe compartirse y debe verificarse su origen antes de proceder con los pasos:\n"
                "paso 1: realizar revisiones\n"
                "paso 2: monitorear sistemas\n"
                "paso 3: Actualizar componentes\n"
                "paso 4: Revisar registros",
                "declaracion.json"
            )
            firma = firmar_documento("declaracion.json", "firma_declaracion.bin")
            print("\nVerificando firma del documento original:")
            verificacion = verificar_firma("declaracion.json", "firma_declaracion.bin")
            print(f"Verificación exitosa: {verificacion}")

        case 2:
            print("\n--- Caso 2: Alterar documento y verificar firma (debe fallar) ---")
            # Asegurarse de que las claves y el documento original existen antes de alterar
            if not os.path.exists("clave_privada.pem") or not os.path.exists("clave_publica.pem") or not os.path.exists("declaracion.json"):
                 print("Error: Asegúrese de ejecutar el Caso 1 primero para generar claves y documento.")
                 return

            print("\nAlterando el documento...")
            with open("declaracion.json", "r") as f:
                documento_alterado = json.load(f)
            documento_alterado["contenido"] = "Este contenido ha sido modificado maliciosamente."
            with open("declaracion.json", "w") as f:
                json.dump(documento_alterado, f, indent=4)
            print("Documento alterado guardado en 'declaracion.json'.")

            print("\nIntentando verificar la firma del documento alterado:")
            verificacion_alterada = verificar_firma("declaracion.json", "firma_declaracion.bin")
            print(f"Verificación exitosa: {verificacion_alterada}")

        case 3:
            print("\n--- Caso 3: Modificar documento después de firmar y verificar firma (debe fallar) ---")
            # Asegurarse de que las claves y el documento original existen antes de modificar
            if not os.path.exists("clave_privada.pem") or not os.path.exists("clave_publica.pem") or not os.path.exists("declaracion.json"):
                 print("Error: Asegúrese de ejecutar el Caso 1 primero para generar claves y documento.")
                 return

            # Asegurarse de que el documento está firmado antes de modificarlo
            if not os.path.exists("firma_declaracion.bin"):
                 print("Error: Asegúrese de firmar el documento primero (ejecute el Caso 1).")
                 return


            print("\n--- Modificando el documento después de la firma ---")
            with open("declaracion.json", "r") as f:
                documento_contenido = json.load(f)

            # Realizar una pequeña modificación en el contenido
            documento_contenido["contenido"] += "\n(este documento ha sido alterado)"

            with open("declaracion.json", "w") as f:
                json.dump(documento_contenido, f, indent=4)

            print("Documento 'declaracion.json' modificado.")

            print("\n--- Verificando la firma del documento modificado ---")
            verificacion = verificar_firma("declaracion.json", "firma_declaracion.bin")
            print(f"Verificación exitosa: {verificacion}")

        case _:
            print("Número de caso no válido.")

# --- Llamada para ejecutar un caso específico ---
# Elija el número de caso (1, 2 o 3) que desea ejecutar
# run_case(1) # descomente para ejecutar el caso 1
# run_case(2) # descomente para ejecutar el caso 2
# run_case(3) # descomente para ejecutar el caso 3

# Ejemplo de ejecución del caso 1:
# run_case(1)

# Ejemplo de ejecución de los casos 1 y 2 secuencialmente:
# run_case(1)
# run_case(2)

# Ejemplo de ejecución de los casos 1, 3 secuencialmente:
run_case(1)
run_case(3)


--- Caso 1: Generar claves, crear documento, firmar y verificar (original) ---
Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.
Documento creado y guardado en: declaracion.json
Documento firmado. Firma guardada en: firma_declaracion.bin
Firma en formato base64 guardada en: firma_declaracion.bin.b64

Verificando firma del documento original:
¡La firma es válida! El documento es auténtico y no ha sido modificado.
Verificación exitosa: True

--- Caso 3: Modificar documento después de firmar y verificar firma (debe fallar) ---

--- Modificando el documento después de la firma ---
Documento 'declaracion.json' modificado.

--- Verificando la firma del documento modificado ---
¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.
Error: 
Verificación exitosa: False
